Authors: Bex Dunn, Neil Symington, Claire Krause, Leo Lymburner

### get some modules

In [18]:
%load_ext autoreload
%autoreload 2

from datetime import datetime
import matplotlib.pyplot as plt

#dealing with system commands
import sys
import os.path

#modules for datacube
import datacube
from datacube.utils import geometry
from datacube.storage.storage import write_dataset_to_netcdf
from datacube.helpers import write_geotiff

# Import external functions from dea-notebooks
sys.path.append(os.path.expanduser('~/dea-notebooks/Scripts/'))
from RainfallTools import load_rainfall, calculate_residual_mass_curve

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Choose a time period and location to calculate residual mass curve.
 - Note: please select whole months here, not days, as a month with only two days may negatively affect your mass residual curve

In [114]:
## Choose a query area and time
##note: datacube is inclusive of both ends of the interval.

query = {
        'lat': (-35.25, -35.35),
        'lon': (149.05, 149.17),
        'time':('2010-01-01', '2012-12-31')
        }

### load rainfall data

In [92]:
#using the BoM grids to 2013. These are out by 2.5km, be aware.

In [93]:
#rainfall = load_rainfall(query)
dc = datacube.Datacube(app='dc-BOMrainfall')
rainfall = dc.load(product = 'bom_rainfall_grids', **query)

## calculate rainfall mass residual

In [94]:
'''This function calculates the residual mass rainfall curve ##FIXME
:param a: '''
import datacube
import numpy as np
import xarray as xr

## resample rainfall data to month start ##FIXME, daily option

In [95]:
#we have daily rainfall here.

In [96]:
rainfall

<xarray.Dataset>
Dimensions:    (latitude: 2, longitude: 3, time: 1096)
Coordinates:
  * time       (time) datetime64[ns] 2010-01-01 2010-01-02 2010-01-03 ...
  * latitude   (latitude) float64 -35.27 -35.32
  * longitude  (longitude) float64 149.1 149.1 149.2
Data variables:
    rainfall   (time, latitude, longitude) float32 1.0589818 1.0485023 ...
Attributes:
    crs:      EPSG:4326

In [97]:
#cumulative_rainfall = rainfall.cumsum(dim='time',keep_attrs=True)

In [98]:
#daily_mean_rainfall = rainfall.groupby('time.dayofyear').mean(dim='time', keep_attrs=True)

In [99]:
#d#aily_mean_rainfall.plot(rainfall, time.dayofyear)

In [100]:
#monthly_mean_rainfall = rainfall.groupby('time.month').mean(dim='time', keep_attrs='True')

In [115]:
#monthly_mean_rainfall.rainfall.plot(col='month',col_wrap=3)

In [102]:
#monthly_mean_rainfall

In [103]:
#rainfall.rainfall.plot(col='time')

In [104]:
#cumulative_rainfall.rainfall.plot(col='time')

In [105]:
#monthly_mean_rainfall.rainfall[-1]

In [106]:
#cumulative_rainfall.rainfall[-1]

In [107]:
## Bex version

In [167]:
#resample rainfall data to month start
monthly_rainfall = rainfall.resample(time='MS').sum('time')

In [168]:
monthly_rainfall

<xarray.Dataset>
Dimensions:    (latitude: 2, longitude: 3, time: 36)
Coordinates:
  * time       (time) datetime64[ns] 2010-01-01 2010-02-01 2010-03-01 ...
  * latitude   (latitude) float64 -35.27 -35.32
  * longitude  (longitude) float64 149.1 149.1 149.2
Data variables:
    rainfall   (time, latitude, longitude) float32 6.088235 7.4755325 ...

In [165]:
monthly_rainfall_2

<xarray.Dataset>
Dimensions:   (month: 12)
Coordinates:
  * month     (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    rainfall  (month) float64 1.289 5.049 4.078 0.7255 1.313 0.9395 1.464 ...

In [156]:
monthly_rainfall.time.dt.month

<xarray.DataArray 'month' (time: 36)>
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,  1,  2,  3,  4,  5,  6,
        7,  8,  9, 10, 11, 12,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])
Coordinates:
  * time     (time) datetime64[ns] 2010-01-01 2010-02-01 2010-03-01 ...

In [134]:
list(monthly_mean_rainfall.groupby('month'))

[(1, <xarray.Dataset>
  Dimensions:    (latitude: 2, longitude: 3)
  Coordinates:
    * latitude   (latitude) float64 -35.27 -35.32
    * longitude  (longitude) float64 149.1 149.1 149.2
      month      int64 1
  Data variables:
      rainfall   (latitude, longitude) float32 1.2327089 1.2016234 1.2202531 ...
  Attributes:
      crs:      EPSG:4326), (2, <xarray.Dataset>
  Dimensions:    (latitude: 2, longitude: 3)
  Coordinates:
    * latitude   (latitude) float64 -35.27 -35.32
    * longitude  (longitude) float64 149.1 149.1 149.2
      month      int64 2
  Data variables:
      rainfall   (latitude, longitude) float32 5.166558 4.991827 4.8559637 ...
  Attributes:
      crs:      EPSG:4326), (3, <xarray.Dataset>
  Dimensions:    (latitude: 2, longitude: 3)
  Coordinates:
    * latitude   (latitude) float64 -35.27 -35.32
    * longitude  (longitude) float64 149.1 149.1 149.2
      month      int64 3
  Data variables:
      rainfall   (latitude, longitude) float32 4.139104 3.9780073 3.

In [178]:
rainfall_monthly_mean_test= rainfall.groupby('time.month').mean(dim=('time'))

In [180]:
rainfall_monthly_mean_test

<xarray.Dataset>
Dimensions:    (latitude: 2, longitude: 3, month: 12)
Coordinates:
  * latitude   (latitude) float64 -35.27 -35.32
  * longitude  (longitude) float64 149.1 149.1 149.2
  * month      (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    rainfall   (month, latitude, longitude) float32 1.2327089 1.2016234 ...

In [181]:
monthly_mean_rainfall.groupby('month').groups

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11}

In [182]:
# find the number of years in the data
years_in_data = len(monthly_rainfall.rainfall.time)/12

In [183]:
years_in_data

3.0

In [145]:
#years_of = monthly_rainfall.groupby('time.year').apply(np.cumsum)

In [184]:
# def divide_by_monthly_ave(in_data):
#     out_month = in_data.cumsum/in_ave
#     return outmonth
    

In [ ]:
# rainfall.groupby('time.month').apply(divide_by_monthly_ave())

In [169]:
#years_of.rainfall.plot(col='time', col_wrap=3)

In [ ]:
# First calculate a cumulative rainfall xarray from the rainfall data

In [185]:
monthly_rainfall

<xarray.Dataset>
Dimensions:    (latitude: 2, longitude: 3, time: 36)
Coordinates:
  * time       (time) datetime64[ns] 2010-01-01 2010-02-01 2010-03-01 ...
  * latitude   (latitude) float64 -35.27 -35.32
  * longitude  (longitude) float64 149.1 149.1 149.2
Data variables:
    rainfall   (time, latitude, longitude) float32 6.088235 7.4755325 ...

In [124]:
cumulative_rainfall = monthly_rainfall.cumsum(dim='time',keep_attrs=True)

In [127]:
cumulative_rainfall_test= rainfall.groupby('time.month')

In [49]:

arr = monthly_rainfall.rainfall.values

cum_rf = np.cumsum(arr, axis = 0)

In [40]:
cum_rf_xr = xr.DataArray(cum_rf, dims = ('time', 'latitude', 'longitude'),
                        coords = [monthly_rainfall.time, monthly_rainfall.latitude, monthly_rainfall.longitude])

In [41]:
cum_rf_xr

<xarray.DataArray (time: 37, latitude: 2, longitude: 3)>
array([[[   6.088235,    7.475533,    8.816736],
        [   4.850801,    6.52524 ,    8.220547]],

       [[ 165.11528 ,  159.44656 ,  157.70885 ],
        [ 169.46277 ,  162.51122 ,  159.57468 ]],

       ...,

       [[2438.5889  , 2377.4202  , 2314.4778  ],
        [2580.275   , 2456.8137  , 2370.8484  ]],

       [[2507.9404  , 2449.1772  , 2387.836   ],
        [2655.5999  , 2534.4304  , 2449.6636  ]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 2010-01-01 2010-02-01 2010-03-01 ...
  * latitude   (latitude) float64 -35.27 -35.32
  * longitude  (longitude) float64 149.1 149.1 149.2

In [ ]:
# NOw we will calculate a cumulative rainfall assuming average rainfall on a month by month basis
# Find the average of all months
ave_months = a.rainfall.groupby('time.month').mean('time').values

# In the case that we are not starting from January we will need to reorder the array

start_month = a.time[0].dt.month.values - 1

ave_month = np.concatenate((ave_months[start_month:,:,:], ave_months[0:start_month,:,:]), axis = 0)


# Tile an array so that we can run a cumulative sum on it
tiled_ave = np.tile(ave_months, (round(n), 1, 1))

# In the case that we have residual months remove them from the tiled array
if (n).is_integer() == False:
    month_remainder = int(round((n%1) * 12))

    tiled_ave = tiled_ave[:int(-month_remainder),:,:]

# Generate the cumulative sum of rainfall one would get assuming average rainfall every month
cum_ave = np.cumsum(tiled_ave, axis = 0)

cum_ave_xr = xr.DataArray(cum_ave, dims = ('time', 'latitude', 'longitude'),
                          coords = [a.time, a.latitude, a.longitude])

# The mass residual curve is the difference between the cumulative rainfall data and the cumulative
# rainfall one would get iff the average always occured
mass_res_curve = cum_rf_xr - cum_ave_xr

return mass_res_curve

In [25]:
# def calculate_residual_mass_curve(a):
#     '''This function calculates the residual mass rainfall curve
#     :param a: '''
    
#     #resample rainfall data to month start
#     a = a.resample('MS', dim='time', how='sum', keep_attrs=True) 
#     # find the number of time steps (ie. years)
#     n = len(a.rainfall.time)/12
    
#     # First calculate a cumulative rainfall xarray from the rainfall data
    
#     arr = a.rainfall.values
    
#     cum_rf = np.cumsum(arr, axis = 0)
    
#     cum_rf_xr = xr.DataArray(cum_rf, dims = ('time', 'latitude', 'longitude'),
#                             coords = [a.time, a.latitude, a.longitude])
    
#     # NOw we will calculate a cumulative rainfall assuming average rainfall on a month by month basis
#     # Find the average of all months
#     ave_months = a.rainfall.groupby('time.month').mean('time').values
   
#     # In the case that we are not starting from January we will need to reorder the array
    
#     start_month = a.time[0].dt.month.values - 1
    
#     ave_month = np.concatenate((ave_months[start_month:,:,:], ave_months[0:start_month,:,:]), axis = 0)

    
#     # Tile an array so that we can run a cumulative sum on it
#     tiled_ave = np.tile(ave_months, (round(n), 1, 1))
    
#     # In the case that we have residual months remove them from the tiled array
#     if (n).is_integer() == False:
#         month_remainder = int(round((n%1) * 12))

#         tiled_ave = tiled_ave[:int(-month_remainder),:,:]
        
#     # Generate the cumulative sum of rainfall one would get assuming average rainfall every month
#     cum_ave = np.cumsum(tiled_ave, axis = 0)
    
#     cum_ave_xr = xr.DataArray(cum_ave, dims = ('time', 'latitude', 'longitude'),
#                               coords = [a.time, a.latitude, a.longitude])
    
#     # The mass residual curve is the difference between the cumulative rainfall data and the cumulative
#     # rainfall one would get iff the average always occured
#     mass_res_curve = cum_rf_xr - cum_ave_xr
    
#     return mass_res_curve


## calculate metrics based on rainfall
- increasing and decreasing periods
- wet year after dry years
- dry year after wet years
-

## calculate metrics based on residual rainfall
- increasing and decreasing periods
- wet year after dry years
- dry year after wet years


## calculate seasonality
- Koppen climate region (use notebook to follow decision tree - requires climate data)
- Summer dominant, Summer, Uniform, Winter, Winter dominant #
- TROP SOI value
- IOD /SAM indices
"During El Niño, rainfall in eastern Australian is typically below average during winter and spring. A neutral ENSO phase has little effect on Australian climate."Bureau of meteorology http://www.bom.gov.au/climate/enso/  

In [39]:
# Calculate failed wet seasons - calculate total wet season + dry season rainfalls
# (end of wet season rainfalls?)

In [30]:
rainfall

<xarray.Dataset>
Dimensions:    (latitude: 2, longitude: 3, time: 1127)
Coordinates:
  * time       (time) datetime64[ns] 2010-01-01 2010-01-02 2010-01-03 ...
  * latitude   (latitude) float64 -35.27 -35.32
  * longitude  (longitude) float64 149.1 149.1 149.2
Data variables:
    rainfall   (time, latitude, longitude) float32 1.0589818 1.0485023 ...
Attributes:
    crs:      EPSG:4326

In [28]:
np.version.version


'1.14.0rc1'

In [ ]:
rainfall.expand_dims()